In [9]:
import pandas as pd
import numpy as np
from pyclustertend import hopkins
import seaborn as sns
import matplotlib.pyplot as plt

file_path = "./movies.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")

df.head()

ModuleNotFoundError: No module named 'pyclustertend'

### Reprocesamiento de datos
Esta parte elimina las columnas irrelevantes y permite el manejar datos nulos

In [8]:

columnas_a_eliminar = [
    "id", "originalTitle", "title", "homePage", "video", "director", 
    "actors", "actorsCharacter", "productionCompany", "productionCompanyCountry", 
    "productionCountry", "releaseDate"
]

columnas_a_eliminar = [col for col in columnas_a_eliminar if col in df.columns]

df.drop(columns=columnas_a_eliminar, inplace=True)

print(df.head())



     budget                            genres      revenue  runtime  \
0   4000000                      Crime|Comedy    4257354.0       98   
1  21000000             Action|Thriller|Crime   12136938.0      110   
2  11000000  Adventure|Action|Science Fiction  775398007.0      121   
3  94000000                  Animation|Family  940335536.0      100   
4  55000000              Comedy|Drama|Romance  677387716.0      142   

                                    actorsPopularity originalLanguage  \
0  22.225|23.519|17.816|19.893|9.027|7.147|7.769|...               en   
1  9.008|6.383|10.757|18.295|11.772|14.777|9.669|...               en   
2  11.881|24.542|14.434|10.651|6.888|5.811|1.432|...               en   
3  9.79|8.084|8.538|33.379|11.733|11.866|8.141|12...               en   
4  33.297|13.246|6.748|6.402|13.189|10.974|10.259...               en   

   popularity  voteAvg  voteCount  genresAmount  productionCoAmount  \
0      20.880      5.7       2077             2                